In [1]:
# (ติดตั้ง library ที่จำเป็น เผื่อยังไม่มี)
import sys
!{sys.executable} -m pip install imbalanced-learn joblib

In [2]:
import pandas as pd
import numpy as np
import os
import joblib # Library สำหรับ Save โมเดล

from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from warnings import simplefilter

# (ปิด Warning ที่ไม่จำเป็น)
simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=UserWarning)

# ---
# ขั้นที่ 1: ตั้งค่าโปรเจกต์
# ---
print("ขั้นที่ 1: กำลังตั้งค่าโปรเจกต์...")

# --- (ไฟล์ Input) ---
RAIN_DATA_PATH = "../data/PROCESSED/rain_2024_with_seasons.csv"
CLUSTER_DATA_PATH = "../data/PROCESSED/master_features_clustered.csv"

# --- (โฟลเดอร์ Output) ---
MODEL_OUTPUT_DIR = "../models"
os.makedirs(MODEL_OUTPUT_DIR, exist_ok=True) # สร้างโฟลเดอร์ models/ ถ้ายังไม่มี
print(f"โมเดลที่เทรนเสร็จแล้วจะถูกเก็บไว้ใน: {MODEL_OUTPUT_DIR}")

# --- (พารามิเตอร์ของโมเดล "ผู้ชนะ") ---
HEAVY_RAIN_THRESHOLD = 5.0 # ใช้เกณฑ์ 5.0 mm
print(f"กำหนดเกณฑ์ 'ฝนหนัก' (y=1) คือ > {HEAVY_RAIN_THRESHOLD} mm")

FEATURES = [
    'lag_1d', 'lag_2d', 'lag_3d', 'lag_4d', 'lag_5d', 'lag_6d', 'lag_7d',
    'rolling_mean_7d', 'rolling_sum_3d',
    'month', 'day_of_week',
    'season_Summer', 'season_Winter'
]
TARGET = 'target_heavy_rain'

print("-" * 50)

# ---
# ขั้นที่ 2: โหลดและเตรียมข้อมูลหลัก
# ---
print("ขั้นที่ 2: กำลังโหลดข้อมูลหลัก (ฝน และ Cluster)...")

# 2.1 โหลดข้อมูลฝนทั้งหมด (เหมือนเดิม)
df_rain_raw = pd.read_csv(RAIN_DATA_PATH)
df_rain_raw['date_object'] = pd.to_datetime(df_rain_raw['date_object'])
df_rain_raw = df_rain_raw.rename(columns={
    'date_object': 'date',
    'dcode': 'district_code',
    'ฝน 24 ชม.': 'rainfall_mm'
})
# เลือกเฉพาะข้อมูลที่จำเป็น
df_data = df_rain_raw[['date', 'district_code', 'rainfall_mm', 'season']].copy()

# 2.2 โหลดข้อมูล Cluster เพื่อเอารายชื่อเขต Cluster 0
df_cluster_raw = pd.read_csv(CLUSTER_DATA_PATH)
df_cluster_raw = df_cluster_raw.rename(columns={'dcode': 'district_code'})
high_risk_districts = df_cluster_raw[df_cluster_raw['cluster'] == 0]['district_code'].unique()

print(f"โหลดข้อมูลสำเร็จ! พบเขตเสี่ยง (Cluster 0) ทั้งหมด {len(high_risk_districts)} เขต")
print("-" * 50)


# ---
# ขั้นที่ 3: วนลูปเทรนโมเดลทั้งหมด
# ---
print("ขั้นที่ 3: เริ่มต้นการเทรนโมเดลแบบ Batch...")

# สร้าง Dictionary ไว้เก็บผลลัพธ์เพื่อเปรียบเทียบ
all_results = {}

# (วนลูป)
for dcode in high_risk_districts:
    print(f"==================================================")
    print(f"กำลังประมวลผล: เขต {dcode}")
    print(f"==================================================")
    
    try:
        # --- (3.1) เตรียมข้อมูลสำหรับเขตนี้ ---
        df_district = df_data[df_data['district_code'] == dcode].copy()
        df_district = df_district.sort_values(by='date').set_index('date')
        df_ts = df_district.resample('D').agg({
            'rainfall_mm': 'mean',
            'season': 'first'
        })
        df_ts['rainfall_mm'] = df_ts['rainfall_mm'].fillna(0)
        df_ts['season'] = df_ts['season'].ffill().bfill()

        # --- (3.2) สร้าง Feature และ Target ---
        df_ts[TARGET] = (df_ts['rainfall_mm'].shift(-1) > HEAVY_RAIN_THRESHOLD).astype(int)
        for i in range(1, 8):
            df_ts[f'lag_{i}d'] = df_ts['rainfall_mm'].shift(i)
        df_ts['rolling_mean_7d'] = df_ts['rainfall_mm'].shift(1).rolling(window=7).mean()
        df_ts['rolling_sum_3d'] = df_ts['rainfall_mm'].shift(1).rolling(window=3).sum()
        df_ts['month'] = df_ts.index.month
        df_ts['day_of_week'] = df_ts.index.dayofweek
        df_ts = pd.get_dummies(df_ts, columns=['season'], drop_first=True)
        
        df_model_data = df_ts.dropna()

        # --- (3.3) ตรวจสอบข้อมูล (สำคัญมาก!) ---
        if len(df_model_data) < 50:
            print("...ข้ามเขตนี้: มีข้อมูลน้อยเกินไป (< 50 วัน)")
            continue
            
        if df_model_data[TARGET].nunique() < 2:
            print("...ข้ามเขตนี้: ไม่พบข้อมูล 'ฝนหนัก' (Class 1) เลย")
            continue

        # --- (3.4) แบ่ง Train/Test (สำหรับเขตนี้) ---
        split_point = int(len(df_model_data) * 0.8)
        df_train = df_model_data.iloc[:split_point]
        df_test = df_model_data.iloc[split_point:]

        X_train = df_train[FEATURES]
        y_train = df_train[TARGET]
        X_test = df_test[FEATURES]
        y_test = df_test[TARGET]
        
        # (ตรวจสอบอีกครั้งว่ามี Class 1 ใน Train Set)
        if y_train.nunique() < 2:
            print("...ข้ามเขตนี้: ไม่พบข้อมูล 'ฝนหนัก' (Class 1) ใน Train Set")
            continue

        # --- (3.5) เทรนโมเดล "ผู้ชนะ" (LR + SMOTE) ---
        pipeline = ImbPipeline([
            ('smote', SMOTE(random_state=42)),
            ('model', LogisticRegression(random_state=42, max_iter=1000))
        ])
        
        pipeline.fit(X_train, y_train)
        
        # --- (3.6) วัดผล ---
        y_pred = pipeline.predict(X_test)
        report = classification_report(y_test, y_pred, output_dict=True, zero_division=0)
        
        # (ดึงค่า Recall (1) มาเก็บ)
        recall_class_1 = report.get('Heavy Rain (1)', {}).get('recall', 0.0)
        all_results[dcode] = recall_class_1
        
        print(f"...เทรนสำเร็จ! Recall (Class 1) ของเขตนี้: {recall_class_1:.2f}")

        # --- (3.7) บันทึกโมเดล! ---
        model_filename = os.path.join(MODEL_OUTPUT_DIR, f"model_dcode_{dcode}.joblib")
        joblib.dump(pipeline, model_filename)
        print(f"...บันทึกโมเดลลงใน '{model_filename}' เรียบร้อย")

    except Exception as e:
        print(f"...เกิดข้อผิดพลาดในเขต {dcode}: {e}")
        continue

print("\n" + "=" * 50)
print("--- การเทรน Batch เสร็จสมบูรณ์ ---")
print("=" * 50)


# ---
# ขั้นที่ 4: สรุปผลลัพธ์
# ---
print("\nขั้นที่ 4: สรุปผลการเทรนของทุกเขต")

df_summary = pd.DataFrame.from_dict(all_results, orient='index', columns=['Recall_Class_1'])
df_summary = df_summary.sort_values(by='Recall_Class_1', ascending=False)

print(df_summary)

print("\n" + "-" * 50)
print(f"ค่า Recall (Class 1) เฉลี่ยของทุกเขตใน Cluster 0: {df_summary['Recall_Class_1'].mean():.2f}")

ขั้นที่ 1: กำลังตั้งค่าโปรเจกต์...
โมเดลที่เทรนเสร็จแล้วจะถูกเก็บไว้ใน: ../models
กำหนดเกณฑ์ 'ฝนหนัก' (y=1) คือ > 5.0 mm
--------------------------------------------------
ขั้นที่ 2: กำลังโหลดข้อมูลหลัก (ฝน และ Cluster)...
โหลดข้อมูลสำเร็จ! พบเขตเสี่ยง (Cluster 0) ทั้งหมด 26 เขต
--------------------------------------------------
ขั้นที่ 3: เริ่มต้นการเทรนโมเดลแบบ Batch...
กำลังประมวลผล: เขต 1045
...เทรนสำเร็จ! Recall (Class 1) ของเขตนี้: 0.00
...บันทึกโมเดลลงใน '../models/model_dcode_1045.joblib' เรียบร้อย
กำลังประมวลผล: เขต 1048
...เทรนสำเร็จ! Recall (Class 1) ของเขตนี้: 0.00
...บันทึกโมเดลลงใน '../models/model_dcode_1048.joblib' เรียบร้อย
กำลังประมวลผล: เขต 1011
...เทรนสำเร็จ! Recall (Class 1) ของเขตนี้: 0.00
...บันทึกโมเดลลงใน '../models/model_dcode_1011.joblib' เรียบร้อย
กำลังประมวลผล: เขต 1019
...เทรนสำเร็จ! Recall (Class 1) ของเขตนี้: 0.00
...บันทึกโมเดลลงใน '../models/model_dcode_1019.joblib' เรียบร้อย
กำลังประมวลผล: เขต 1044
...เทรนสำเร็จ! Recall (Class 1) ของเขตนี้: 0.00
...บั